# Curso: Machine Learning para Pronóstico Hidrológico

<div style="text-align:center;">
    <img src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/Logo_course.png?raw=true" width="300">
    <p style="margin-top:10px;">
        Contacto: paul.andres.munoz@gmail.com
    </p>
    <p><a href="https://paulmunozpauta.github.io/paulmunozpauta/index.html" target="_blank">Website personal</a></p>
</div>




## ✅ Antes de comenzar: sigue estos 4 pasos para ejecutar el notebook en Google Colab


Paso 1. Clona el repositorio de GitHub con los notebooks y datos del curso.


In [ ]:
!git clone -- https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico.git

Paso 2. Accede a la carpeta clonada.


In [ ]:
ls

In [ ]:
%cd Curso_ML_pronostico_hidrologico

In [ ]:
ls

Paso 3. Configura el entorno para ejecutar el código del curso.


In [ ]:
# Resolver un conflicto con Colab, nueva versión de numpy
!pip uninstall -y numpy
# Instalar las versiones de numpy para el curso
!pip install numpy==1.24.4

In [ ]:
# Install Poetry
!pip install poetry
# Disable virtual environment creation (needed for Colab)
!poetry config virtualenvs.create false

🔁 Si la sesión se reinicia, repite los pasos 2 y 3.
➡️ Si no, continúa con el paso 4.


Paso 4. Instala los paquetes necesarios para el curso.


In [ ]:
%cd Curso_ML_pronostico_hidrologico
!poetry lock


In [ ]:
!poetry install --no-root

🧪 Ahora sí, empezamos con la parte práctica del curso


# 🛰️ Parte 1: Adquisición y preprocesamiento de datos de precipitación satelital

En esta sesión aprenderemos a:
- Acceder y descargar datos de dos fuentes globales de precipitación satelital:
  * PERSIANN-CCS (Universidad de California)
  * IMERG-ER (NASA - misión GPM)
- Abrir, georreferenciar y visualizar datos de precipitación satelital para una región específica.

# 🧰 Software para acceder a repositorios gratuitos de datos satelitales

## Descargar el software
https://filezilla-project.org/download.php?type=client

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/FileZilla_download.png?raw=1"></div>

## 🔑 Iniciar FileZilla

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/FileZilla.png?raw=1"></div>

# 🌐 Fuente de datos PERSIANN

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/PERSIANN_head.png?raw=1"></div>

Website: http://chrs.web.uci.edu

Subproductos disponibles:
- PERSIANN
- PERSIANN-CCS
- PERSIANN-CDR
- PERSIANN-IDR

Nos enfocaremos en el subproducto con la resolución espacial/temporal más fina: **PERSIANN-CCS**.

¿Qué es PERSIANN-CCS?

PERSIANN-CCS utiliza imágenes satelitales para analizar las nubes y estimar la precipitación a nivel global. Fue desarrollado por la Universidad de California en Irvine.

El método de estimación de precipitación es muy detallado, analizando la altura de las nubes y su cobertura. A diferencia de otros métodos, puede identificar nubes individuales y proporcionar información específica sobre ellas.
Esto permite estimar con más precisión cuánta precipitación cae en diferentes regiones, en tiempo real y con posibilidad de descarga gratuita.

📝 En resumen:
- **Período de datos**: Desde 2003 hasta el presente
- **Cobertura**: Desde 60°S hasta 60°N
- **Resolución**: 0.04° x 0.04° (~4.4 x 4.4 km por píxel)
- **Disponibilidad temporal**: Cada 1h, 3h, 6h, diario, mensual y anual
- **Latencia**: Prácticamente en tiempo real (~2 hours)

## Visualizar datos en la plataforma
Accede a la plataforma de visualización de datos usando el siguiente enlace:
[https://chrsdata.eng.uci.edu](https://chrsdata.eng.uci.edu)

## Conectar al repositorio PERSIANN Data

Ingresar la siguiente información:

- **Protocol**: FTP - File Transfer Protocol  
- **Host**: persiann.eng.uci.edu  
- **Logon Type**: Anonymous  

<div style="text-align:center;"><img style="width: 60%;" src="https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/FTP_PERSIANN.png?raw=1"></div>

Una vez conectado, tendrás acceso remoto a la carpeta de datos PERSIANN.



<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/Filezilla_PERSIANN_CCS.png?raw=1"></div>

## Descargar datos del repositorio PERSIANN-CCS









Click derecho en la carpeta/archivo que queremos descargar en la carpeta local

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/Filezilla_local.png?raw=1"></div>

Ahora tenemos los archivos de precipitación PERSIANN-CCS.









## Abrir un archivo PERSIANN-CCS









*Importar* las librerías necesarias:









In [ ]:
import gzip
import descartes
import pickle
import numpy as np
from shapely.geometry import mapping
import geopandas as gpd
import os
import glob
import rasterio
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import datetime
import calendar
import pandas as pd
from scipy import signal
import pickle
import h5py
def JulianDate_to_MMDDYYY(y,jd):
    month = 1
    day = 0
    while jd - calendar.monthrange(y,month)[1] > 0 and month <= 12:
        jd = jd - calendar.monthrange(y,month)[1]
        month = month + 1
    return jd, month

def getLine(data, line_no):
    n = 0
    lastPos = -1
    for i in range(0, len(data) - 1):
        if data[i] == "\n":
            n = n + 1
            if n == line_no:
                return data[lastPos + 1:i]
            else:
                lastPos = i;

    if(n == line_no - 1):
        return data[lastPos + 1:]
    return ""

Define la carpeta del proyecto donde se guardan los datos de PERSIANN-CCS.











In [ ]:
folder = os.getcwd()
folder

In [ ]:
folder_files= folder+'/notebooks/data/PERSIANN-CCS/Hourly/Global/2023/'
folder_files

Define el nombre de un archivo para su lectura

In [ ]:
item=folder_files+'rgccs1h2301500.bin.gz'
item

'rgccs1h2301500.bin.gz' es un archivo binario comprimido cuyo nombre codifica lo siguiente:

**1h**: datos horarios

**23**: año 2023

**015**: día 15 del año

**00**: datos correspondientes a las 00 horas (formato de 24 horas)

Leer el archivo

In [ ]:
f = gzip.GzipFile(item)
file_content = f.read()
data = np.frombuffer(file_content, dtype=np.dtype('>h')).astype(float)
data = data.reshape((3000,9000))
data_1 = data[:,4500:]
data_2 = data[:,:4500]
data = np.hstack((data_1,data_2))
data= data/100
data[data < 0] = np.nan
data = np.flipud(data)

In [ ]:
data

Georreferenciación de la matriz de datos.









In [ ]:
lon=np.arange(-180,180,0.04)
lat=np.arange(60,-60,-0.04)
data = xr.DataArray(data=data, dims=["lat", "lon"], coords=[lat,lon])
data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data

Graficar el archivo de precipitación global georreferenciado.









In [ ]:
# Load the world map correctly
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# Create figure and axis
fig, ax = plt.subplots(figsize=(10, 10))
# Plot world boundaries
world.boundary.plot(ax=ax, color='white', linewidth=0.5)
# Plot the data
max_val = data.max()
im = ax.imshow(data, cmap='cividis', vmin=0.1, vmax=int(max_val), extent=[-180, 180, -90, 90])
# Set aspect ratio
ax.set_aspect('equal')
# Add title and labels
plt.title("Global precipitation PERSIANN-CCS")
cbar = plt.colorbar(im, ax=ax, orientation='horizontal')
cbar.set_label('Precipitation (mm)')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
# Show the plot
plt.show()

Extraer estadísticas básicas de los datos

In [ ]:
print('Maximum precipitation = ',data.max().values,'mm')
print('Average precipitation = ',data.mean().values,'mm')
print('Minimum precipitation = ',data.min().values,'mm')

Cargar el shapefile de un sistema hidrológico:
- Por ejemplo, una cuenca montañosa en Ecuador, Sudamérica

In [ ]:
data.rio.write_crs("epsg:4326", inplace=True)
catchment_shp_1 = gpd.read_file(folder+'/notebooks/shapefiles/Catchment_SA.shp')

Mostrar la cuenca

In [ ]:
catchment_shp_1.plot()

In [ ]:
import folium
# Convert to WGS 84 (if needed)
if catchment_shp_1.crs != "EPSG:4326":
    catchment_shp_1 = catchment_shp_1.to_crs(epsg=4326)
centroid = catchment_shp_1.geometry.centroid.iloc[0]
map_center = [centroid.y, centroid.x]
# Create a folium map centered on the catchment area
m = folium.Map(location=map_center, zoom_start=10)
# Add the catchment boundaries to the map
folium.GeoJson(catchment_shp_1, name="Catchment").add_to(m)
# Display the map
m

Área de la cuenca (polígono).









In [ ]:
catchment_shp_1 = catchment_shp_1.to_crs(epsg=32717)
catchment_shp_1["Area_m2"] = catchment_shp_1.geometry.area
catchment_shp_1["Area_km2"] = catchment_shp_1["Area_m2"] / 1e6
catchment_shp_1["Area_km2"]

Recortar los datos de precipitación global a la cuenca









In [ ]:
data_catchment_1 = data.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)

Mostrar la precipitación de la cuenca de montaña

In [ ]:
fig = plt.subplots(figsize=(8,8))
data_catchment_1.plot(cmap='Blues')
plt.title("PERSIANN-CCS precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

Estadísticas básicas de precipitación para la cuenca









In [ ]:
print('Maximum precipitation = ',data_catchment_1.max().values,'mm')
print('Average precipitation= ',data_catchment_1.mean().values,'mm')
print('Minimum precipitation = ',data_catchment_1.min().values,'mm')

Ahora, para otra cuenca
- Cuenca costera en Ecuador.

In [ ]:
catchment_shp_2 = gpd.read_file(folder+'/notebooks/shapefiles/Coastal_catchment.geojson')
catchment_shp_2.plot()

Echemos un vistazo a la cuenca en Sudamérica.

In [ ]:
# Convert to WGS 84 (if needed)
if catchment_shp_2.crs != "EPSG:4326":
    catchment_shp_2 = catchment_shp_2.to_crs(epsg=4326)
centroid = catchment_shp_2.geometry.centroid.iloc[0]
map_center = [centroid.y, centroid.x]
# Create a folium map centered on the catchment area
m = folium.Map(location=map_center, zoom_start=10)
# Add the catchment boundaries to the map
folium.GeoJson(catchment_shp_2, name="Catchment").add_to(m)
# Display the map
m

Área de la cuenca (polígono)

In [ ]:
# Convert to the correct UTM Zone (update based on location), reprojection for accurate calculations
catchment_shp_2 = catchment_shp_2.to_crs(epsg=32717)  # UTM Zone 17S (Western Ecuador)
catchment_shp_2["Area_m2"] = catchment_shp_2.geometry.area
catchment_shp_2["Area_km2"] = catchment_shp_2["Area_m2"] / 1e6
catchment_shp_2["Area_km2"]

Recortar los datos de precipitación global a la cuenca costera









In [ ]:
data_catchment_2 = data.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)

Mostrar la precipitación en la cuenca

In [ ]:
fig = plt.subplots(figsize=(8,8))
data_catchment_2.plot(cmap='Blues')
plt.title("PERSIANN-CCS precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

Estadísticas básicas de precipitación para la cuenca costera

In [ ]:
print('Maximum precipitation = ',data_catchment_2.max().values,'mm')
print('Average precipitation= ',data_catchment_2.mean().values,'mm')
print('Minimum precipitation = ',data_catchment_2.min().values,'mm')

**Ahora lee todos los archivos de precipitación descargados en la carpeta**

Crear una lista de los archivos descargados









In [ ]:
folder_files
file_extension = "*.bin.gz"
list_of_Files = glob.glob(os.path.join(folder_files, file_extension))
list_of_Files = [file_name for file_name in list_of_Files if not file_name.startswith('.DS_Store')]
list_of_Files.sort()

Mostrar la lista

In [ ]:
list_of_Files[:10]

Leer los archivos, uno por uno, y calcular la precipitación acumulada. La carpeta de datos contiene los primeros 15 días de enero de 2023

In [ ]:
data_sum = xr.DataArray(data=np.empty((3000, 9000)), dims=["lat", "lon"])
for index, item in enumerate(list_of_Files):
    print(index+1, 'out of', len(list_of_Files))
    f=gzip.GzipFile(item)
    try:
        file_content = f.read()
    except (IOError, EOFError) as e:
        continue
    data = np.frombuffer(file_content, dtype=np.dtype('>h')).astype(float)
    data = data.reshape((3000,9000))
    data_1 = data[:,4500:]
    data_2 = data[:,:4500]
    data = np.hstack((data_1,data_2))
    data= data/100
    data[data < 0] = np.nan
    data = np.flipud(data)
    data = xr.DataArray(data=data, dims=["lat", "lon"], coords=[lat,lon])
    data_sum+=data

In [ ]:
data_sum

Georreferenciar los datos

Recortar la precipitación satelital acumulada al shapefile de la cuenca (cuenca montañosa).

In [ ]:
data_sum.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_sum.rio.write_crs("epsg:4326", inplace=True)
data_catchment_1 = data_sum.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)

Graficar los datos.

Mostrar estadísticas básicas.

In [ ]:
fig = plt.subplots(figsize=(8,8))
plt.imshow(data_catchment_1,cmap='cividis',vmin=0.1,vmax=150)
plt.title("PERSIANN-CCS precipitation")
plt.colorbar(label='Precipitation (mm)')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitación = ',data_catchment_1.max().values,'mm')
print('Average precipitación = ',data_catchment_1.mean().values,'mm')
print('Minimum precipitación = ',data_catchment_1.min().values,'mm')

Ahora para la cuenca costera



In [ ]:
data_catchment_2 = data_sum.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)
fig = plt.subplots(figsize=(8,8))
plt.imshow(data_catchment_2,cmap='cividis',vmin=0.1,vmax=150)
plt.title("PERSIANN-CCS precipitation")
plt.colorbar(label='Precipitation (mm)')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitación = ',np.round(data_catchment_2.max().values,2),'mm')
print('Average precipitación = ',np.round(data_catchment_2.mean().values,2),'mm')
print('Minimum precipitación = ',np.round(data_catchment_2.min().values,2),'mm')

## Generar un DataFrame con series temporales de precipitación (para cada píxel):

Ejemplo: En la cuenca de montaña, series temporales para el 1 de enero de 2023.

In [ ]:
dataset_list = []  # Store data frames for concatenation later
for index, item in enumerate(list_of_Files[:24]):
    print(index + 1, 'out of', len(list_of_Files[:24]))

    try:
        with gzip.GzipFile(item, 'rb') as f:
            file_content = f.read()
    except (IOError, EOFError):
        continue  # Skip to next file if there's an error

    data = np.frombuffer(file_content, dtype=np.dtype('>h')).astype(float)  # Convert to float
    data = data.reshape((3000, 9000))
    # Splitting and rearranging data
    data_1 = data[:, 4500:]
    data_2 = data[:, :4500]
    data = np.hstack((data_1, data_2))
    data = data / 100  # Scale values
    data[data < 0] = np.nan  # Replace negative values with NaN
    data = np.flipud(data)  # Flip vertically
    # Convert to xarray DataArray
    data = xr.DataArray(data=data, dims=["lat", "lon"], coords=[lat, lon])
    data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    data.rio.write_crs("epsg:4326", inplace=True)
    # Clip the data using the catchment shape
    data = data.rio.clip(catchment_shp_1.geometry.apply(mapping), catchment_shp_1.crs, all_touched=True)
    data = data.values.flatten()
    # Extract date from filename
    date_att = str(item)
    year = int('20' + str(date_att[-14:-12]))
    julian_day = int(str(date_att[-12:-9]))
    day, month = JulianDate_to_MMDDYYY(year, julian_day)
    hour = int(str(date_att[-9:-7]))
    date = datetime.datetime(year, month, day, hour, 0, 0)
    # Convert data to DataFrame
    data = pd.DataFrame(data)
    data = data.dropna()
    data = data.T
    data.index = pd.DatetimeIndex([date])  # Set datetime index
    dataset_list.append(data)  # Append to the list
# Concatenate all DataFrames
if dataset_list:
    dataset = pd.concat(dataset_list)

Preprocesar la información

Eliminar datos duplicados

In [ ]:
dataset.shape

In [ ]:
dataset = dataset.sort_index()
dataset = dataset[~dataset.index.duplicated(keep='first')]
dataset


Exportar esta información en formato CSV.





In [ ]:
dataset.to_csv(folder_files+"PERSIANN-CCS_UTC_Catchment_1.csv", index=True)

Graficar la precipitación satelital extraída.






In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.plot(kind='bar', ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Satellite precipitation (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=6)
plt.show()

Calcular y graficar la precipitación acumulada.









In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.cumsum().plot(ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Accumulated precipitation (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.3), ncol=6)
plt.show()

## Ejercicio: Generar un DataFrame con series temporales de precipitación (para cada píxel) para la cuenca en la zona costera

# Precipitación satelital IMERG.

Sitio web: https://gpm.nasa.gov/data/imerg

## Subproductos disponibles:

- Early Run
- Late Run
- Final Run

Nos enfocaremos en el subproducto con la resolución espacial/temporal más fina, IMERG-Early Run.

Visualización de datos on-line
https://giovanni.gsfc.nasa.gov/giovanni/

## Registro previo para descargar datos IMERG.

Para acceder y descargar datos IMERG, se requiere un registro previo. Sigue estos pasos:

https://registration.pps.eosdis.nasa.gov/registration/newContact.html

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/IMERG_registration.png?raw=1"></div>


Una vez registrado, confirma tu cuenta mediante el enlace de verificación enviado por correo electrónico.

## Conectarse al repositorio de datos IMERG.

Ingresa la siguiente información para conectarte al servidor FTP de IMERG:

- Protocolo: FTP - Protocolo de Transferencia de Archivos

- Host: jsimpsonftps.pps.eosdis.nasa.gov

- Tipo de inicio de sesión: Normal

- Ingresa tu nombre de usuario y contraseña(igual al correo) de tu cuenta de NASA Earthdata.

Una vez conectado, navega hasta el directorio deseado para acceder a los datos IMERG

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta/Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/IMERG_FTP.png?raw=1"></div>

Al conectarte, obtienes acceso remoto a la carpeta de datos IMERG.
Desde allí, puedes navegar por los directorios para localizar y descargar los conjuntos de datos deseados.

<div style="text-align:center;"><img style="width: 100%;" src="https://github.com/paulmunozpauta//Curso_ML_pronostico_hidrologico/blob/main/notebooks/static/imgs/IMERG_server.png?raw=1"></div>

## Descargar datos del repositorio IMERG-Early Run


Haz clic derecho sobre la carpeta o archivo que deseas descargar y selecciona "Download" para guardarlo en tu directorio local.  

## Abrir un archivo de precipitación IMERG.

Definir la carpeta de datos IMERG-ER

In [ ]:
folder = os.getcwd()
folder_files= folder+'/notebooks/data/IMERG-ER/Hourly/Global/202301/'
folder_files

Crear una lista de archivos de precipitación. Los datos contienen precipitación satelital del 31 de enero de 2023.

In [ ]:
list_of_Files = glob.glob(folder_files + '/*H5')
list_of_Files.sort()

Mostrar la lista

In [ ]:
list_of_Files

Definir la ruta de un archivo para su lectura.









In [ ]:
item=folder_files+'3B-HHR-E.MS.MRG.3IMERG.20230131-S000000-E002959.0000.V06C.RT-H5'
item

La codificación del nombre '3B-HHR-E.MS.MRG.3IMERG.20230131-S000000-E002959.0000.V06C.RT-H5' indica:

- 3B-HHR-E.MS.MRG.3IMERG: información del producto

- 20230131: AñoMesDía

- S000000: inicio del escaneo (hhmmss)

- E002959: fin del escaneo (hhmmss)

Abrir un único archivo de precipitación.









In [ ]:
data = h5py.File( item, 'r' )
precip = data['/Grid/precipitationCal'][:]
precip = np.flip( precip[0,:,:].transpose(), axis=0 )

Mostrar la matriz de datos leída..









In [ ]:
precip

Estadísticas básicas.









In [ ]:
print('Maximum precipitation = ',precip.max(),'mm')
print('Average precipitation = ',precip.mean(),'mm')
print('Minimum precipitation = ',precip.min(),'mm')

Mostrar la información de precipitación global.









In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(precip, vmin=-1, vmax=80, extent=[-180, 180, -90, 90])
cbar = plt.colorbar(im, orientation='horizontal')
cbar.set_label('millimeters/hour')
for lon in np.arange(-90, 90+1, 90):
    plt.plot((lon, lon), (-90, +90), color="black", linewidth=1)
for lat in np.arange(-60, 60+1, 30):
    plt.plot((-180, +180), (lat, lat), color="black", linewidth=1)
plt.show()

Georreferenciar y recortar los datos de precipitación global a la cuenca montañosa.









In [ ]:
precip

In [ ]:
lat = data['/Grid/lat'][:]
lon = data['/Grid/lon'][:]
catchment_shp_1 = gpd.read_file(folder+'/notebooks/shapefiles/Catchment_SA.shp')
data = xr.DataArray(data=precip, dims=["lat", "lon"], coords=[lat,lon])
data.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data.rio.write_crs("epsg:4326", inplace=True)
data_catchment_1= data.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)

Graficar la precipitación para la cuenca montañosa.









In [ ]:
fig = plt.subplots(figsize=(8,8))
data_catchment_1.plot(cmap='cividis',vmin=0,vmax=0.01)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()

Estadísticas básicas.









In [ ]:
print('Maximum precipitation = ', np.round(data_catchment_1.max().values,2),'mm')
print('Average precipitation = ', np.round(data_catchment_1.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_catchment_1.min().values,2),'mm')

Graficar la precipitación para la cuenca costera.









In [ ]:
catchment_shp_2 = gpd.read_file(folder+'/notebooks/shapefiles/Coastal_catchment.geojson')


In [ ]:
data

In [ ]:
data_catchment_2 = data.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)


In [ ]:
fig = plt.subplots(figsize=(8,8))
data_catchment_2.plot(cmap='cividis',vmin=0,vmax=5)
plt.title("Precipitación Global IMERG-ER")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitation = ', np.round(data_catchment_2.max().values,2),'mm')
print('Average precipitation = ', np.round(data_catchment_2.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_catchment_2.min().values,2),'mm')

Leer y procesar todos los archivos IMERG-ER en la carpeta.









Mostrar la lista

In [ ]:
list_of_Files

Acumular precipitación.









In [ ]:
data_sum = xr.DataArray(data=np.empty((1800, 3600)), dims=["lat", "lon"])
for index, item in enumerate(list_of_Files):
    print(index+1, 'out of', len(list_of_Files))
    try:
        data = h5py.File(item, 'r')
        precip = data['/Grid/precipitationCal'][:]
        precip[precip < 0] = np.nan
        precip = np.flip(precip[0,:,:].transpose(), axis=0)
        theLats = data['Grid/lat'][:]
        theLons = data['Grid/lon'][:]
        x, y = np.meshgrid(theLons, theLats)

        precip = xr.DataArray(precip, dims=('lat', 'lon'), coords={'lat' : theLats, 'lon' : theLons})

        data_sum += precip

        data.close()

    except (IOError, EOFError) as e:
        continue

Graficar la información de precipitación acumulada del 31 de enero de 2023.









In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
im = ax.imshow(data_sum, vmin=-1, vmax=1400, extent=[-180, 180, -90, 90])
cbar = plt.colorbar(im, orientation='horizontal')
cbar.set_label('millimeters/hour')

for lon in np.arange(-90, 90+1, 90):
    plt.plot((lon, lon), (-90, +90), color="black", linewidth=1)
for lat in np.arange(-60, 60+1, 30):
    plt.plot((-180, +180), (lat, lat), color="black", linewidth=1)

plt.show()

Recortar esa información a la cuenca montañosa.


In [ ]:
data_sum.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_sum.rio.write_crs("epsg:4326", inplace=True)
data_sum_catchment_1= data_sum.rio.clip(catchment_shp_1.geometry.apply(mapping),catchment_shp_1.crs,all_touched=True)
data_sum_catchment_1

In [ ]:
fig = plt.subplots(figsize=(8,8))
data_sum_catchment_1.plot(cmap='cividis',vmin=0,vmax=16)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitación = ', np.round(data_sum_catchment_1.max().values,2),'mm')
print('Average precipitación = ', np.round(data_sum_catchment_1.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_sum_catchment_1.min().values,2),'mm')

Ahora para la cuenca costera.









In [ ]:
data_sum = xr.DataArray(data_sum, dims=('lat', 'lon'), coords={'lat' : theLats, 'lon' : theLons})
data_sum.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
data_sum.rio.write_crs("epsg:4326", inplace=True)
data_sum_catchment_2= data_sum.rio.clip(catchment_shp_2.geometry.apply(mapping),catchment_shp_2.crs,all_touched=True)
fig = plt.subplots(figsize=(8,8))
data_sum_catchment_2.plot(cmap='cividis',vmin=0,vmax=9)
plt.title("IMERG-ER precipitation")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()
print('Maximum precipitation = ', np.round(data_sum_catchment_2.max().values,2),'mm')
print('Average precipitation = ', np.round(data_sum_catchment_2.mean().values,2),'mm')
print('Minimum precipitation = ', np.round(data_sum_catchment_2.min().values,2),'mm')

Generar una base de datos con series temporales de precipitación (píxeles) para la cuenca montañosa.





In [ ]:
dataset_list = []  # List to store DataFrames for concatenation later
for index, item in enumerate(list_of_Files):
    print(index + 1, 'out of', len(list_of_Files))
    try:
        with h5py.File(item, 'r') as data:
            precip = data['/Grid/precipitationCal'][:]
    except (IOError, EOFError):
        continue  # Skip file if there's an error
    precip[precip < 0] = np.nan  # Replace negative values with NaN
    precip = np.flip(precip[0, :, :].transpose(), axis=0)  # Flip and transpose
    # Convert to xarray DataArray
    precip = xr.DataArray(precip, dims=('lat', 'lon'), coords={'lat': theLats, 'lon': theLons})
    precip.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    precip.rio.write_crs("epsg:4326", inplace=True)
    # Clip the data using the catchment shape
    precip = precip.rio.clip(catchment_shp_1.geometry.apply(mapping), catchment_shp_1.crs, all_touched=True)
    precip = precip.values.flatten()
    # Extract date from filename
    year = int(item[-40:-36])
    month = int(item[-36:-34])
    day = int(item[-34:-32])
    hour = int(item[-22:-20])
    minute = int(item[-20:-18])
    date = datetime.datetime(year, month, day, hour, minute, 0)
    # Convert data to DataFrame
    data = pd.DataFrame(precip)
    data = data.dropna()
    data = data.T
    data.index = pd.DatetimeIndex([date])  # Set datetime index
    dataset_list.append(data)  # Append DataFrame to list
# Concatenate all DataFrames at the end
if dataset_list:
    dataset = pd.concat(dataset_list)
# Resample dataset, treating NaNs as zeros
dataset = dataset.resample('h', label='right', closed='right').sum().fillna(0)

Mostrar dataframe

In [ ]:
dataset

Exportar el dataframe a CSV.









In [ ]:
dataset.to_csv(folder_files+"IMERG-ER_UTC_Catchment_1.csv", index=True)

Graficar la serie temporal de precipitación.









In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.plot(kind='bar', ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=7)
plt.show()

Graficar la precipitación acumulada.









In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
dataset.cumsum().plot( ax=ax)
ax.legend(title='Legend Title')
plt.ylabel('Precipitation_satellite (mm)')
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.6), ncol=7)
plt.show()

## Ejercicio: Hacer lo mismo para la cuenca costera.









Generar una base de datos con series temporales de precipitación (píxeles) para la cuenca montañosa.

Mostrar el dataframe

Exportar el dataframe a CSV.









Graficar la serie temporal de precipitación.









Graficar la precipitación acumulada.







